# YOLOv5 Model Assisted Labeling

> Work based on https://models.roboflow.com/object-detection/yolov5.

Labeling a new dataset from scratch is tiresome. To label images faster, I train a YOLOv5 model and apply its predictions to new images.

## How to use

### Vanilla image sets

In [ ]:
%%capture
#hide
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g6ZdbI07paFW9aen6yylIf5tVe_uc4oHki9Q2mYASsITrHAeoakSk8


In [ ]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"
%run '_Synch.ipynb'

/content/drive/MyDrive/Coding/ModelAssistedLabel
wrapping train.py
Converted 00_ultralytics.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted index.ipynb.
converting: /content/drive/MyDrive/Coding/ModelAssistedLabel/01_split.ipynb
converting: /content/drive/MyDrive/Coding/ModelAssistedLabel/02_train.ipynb
converting: /content/drive/MyDrive/Coding/ModelAssistedLabel/00_ultralytics.ipynb
converting: /content/drive/MyDrive/Coding/ModelAssistedLabel/index.ipynb
converting /content/drive/MyDrive/Coding/ModelAssistedLabel/index.ipynb to README.md
Executing: git config --local include.path ../.gitconfig
Success: hooks are installed and repo's .gitconfig is now trusted


Recursively search a folder (`repo`) that contains images and labels.

In [ ]:
from ModelAssistedLabel.core import Generation, Defaults
g = Generation(repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)", 
              out_dir = ".",
              data_yaml=Defaults().data_yaml)

def tostr(split):
  return [{k: len(v)} for k,v in split.items()]

def sumg(split):
  return sum([list(x.values())[0] for x in tostr(split)])

for MAX_SIZE in [10, None, 5]:
  g.set_split(MAX_SIZE=MAX_SIZE)
  print("summary: ", tostr(g.split))
  print("checksum:", sumg(g.split))

summary:  [{'train': 7}, {'valid': 2}, {'test': 1}]
checksum: 10
summary:  [{'train': 589}, {'valid': 169}, {'test': 83}]
checksum: 841
summary:  [{'train': 4}, {'valid': 1}, {'test': 0}]
checksum: 5


Writing these files to disk for when we need them later.

In [ ]:
zipped = g.process_split(" vanilla")

target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/digittake-298-jpg_jpg.rf.aa1765ea20334421e034081325f11506.jpg | ./train/images/digittake-298-jpg_jpg.rf.aa1765ea20334421e034081325f11506.jpg
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/labels/digittake-298-jpg_jpg.rf.aa1765ea20334421e034081325f11506.txt | ./train/labels/digittake-298-jpg_jpg.rf.aa1765ea20334421e034081325f11506.txt
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/digittake-113-jpg_jpg.rf.23d093be40cc2c1ad6b2289c9b93cb5c.jpg | ./train/images/digittake-113-jpg_jpg.rf.23d093be40cc2c1ad6b2289c9b93cb5c.jpg
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/labels/digittake-113-jpg_jpg.rf.23d093be40cc2c1ad6b2289c9b93cb5c.txt | ./train/labels/digittake-113-jpg_jpg.rf.23d093be40cc2c1ad6b2289c9b93cb5c.txt
target/dest /content/drive/MyDrive/Coding/Roboflow Export (841)/images/digittake-19-jpg_jpg.rf.337c031612641985884e1ad0ec592d2b.jpg | ./train/images/dig

In [ ]:
import os

local = os.path.basename(zipped)
!unzip "{local}" #grab data
!rm -f -r '{local}' #remove zip file

Archive:  Roboflow Export (841) vanilla 21-03-15 21-36-20.zip
   creating: Roboflow Export (841) vanilla 21-03-15 21-36-20/
   creating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/
   creating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/images/
  inflating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/images/digittake-298-jpg_jpg.rf.aa1765ea20334421e034081325f11506.jpg  
  inflating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/images/digittake-113-jpg_jpg.rf.23d093be40cc2c1ad6b2289c9b93cb5c.jpg  
  inflating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/images/digittake-19-jpg_jpg.rf.337c031612641985884e1ad0ec592d2b.jpg  
  inflating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/images/digittake-228-jpg_jpg.rf.8e675a68295352b621f954a1c77848f4.jpg  
   creating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/labels/
  inflating: Roboflow Export (841) vanilla 21-03-15 21-36-20/train/labels/digittake-298-jpg_jpg.rf.aa1765ea

In [ ]:
for splt in ["test/", "train/", "valid/", "data.yaml"]:
  os.system(f"mv '{local[:-4]}/{splt}' .")
os.system(f"rm -f -r '{local[:-4]}'")

In [ ]:
!nbdev_build_lib

Converted 00_ultralytics.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted index.ipynb.


In [ ]:
from ModelAssistedLabel.core import Trainer

In [ ]:
t= Trainer(name="testfromindex")

In [ ]:
%%time
t.train(2)

CPU times: user 1.69 ms, sys: 3.35 ms, total: 5.05 ms
Wall time: 10.5 s


In [ ]:
ls yolov5/runs/train

bingo/  custom/  custom2/  custom3/
